<a href="https://colab.research.google.com/github/jarodchristiansen/PythonProjects/blob/master/News_Text_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vaex
!pip install Ipython --upgrade

Requirement already up-to-date: Ipython in /usr/local/lib/python3.6/dist-packages (7.15.0)


In [ ]:
import pandas as pd
import numpy as np
import vaex
import spacy
import h5py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = vaex.open('/content/drive/My Drive/Colab Notebooks/Neural Networks/NLP/News NLP/all-the-news-2-1.csv_chunk0.hdf5')

In [ ]:
data

#,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication
0,0,0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,'We should take concerns about the health of lib...,"'This post is part of Polyarchy, an independent ...",'https://www.vox.com/polyarchy/2016/12/9/1389834...,None,Vox
1,1,1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,"""Colts GM Ryan Grigson says Andrew Luck's contra...",' The Indianapolis Colts made Andrew Luck the hi...,'https://www.businessinsider.com/colts-gm-ryan-g...,None,Business Insider
2,2,2,2018-01-26 00:00:00,2018,1.0,26,None,Trump denies report he ordered Mueller fired,"'DAVOS, Switzerland (Reuters) - U.S. President D...",'https://www.reuters.com/article/us-davos-meetin...,Davos,Reuters
3,3,3,2019-06-27 00:00:00,2019,6.0,27,None,"""France's Sarkozy reveals his 'Passions' but ins...",'PARIS (Reuters) - Former French president Nicol...,'https://www.reuters.com/article/france-politics...,World News,Reuters
4,4,4,2016-01-27 00:00:00,2016,1.0,27,None,"""Paris Hilton: Woman In Black For Uncle Monty's ...",'Paris Hilton arrived at LAX Wednesday dressed t...,'https://www.tmz.com/2016/01/27/paris-hilton-mon...,None,TMZ
...,...,...,...,...,...,...,...,...,...,...,...,...
"124,995",124995,124995,2019-02-14 00:00:00,2019,2.0,14,"Corina Pons, Luc Cohen, Matt Spetalnick","""Venezuela's Maduro ramps up legal fight against...",'CARACAS/WASHINGTON (Reuters) - Venezuelan Presi...,'https://www.reuters.com/article/us-venezuela-po...,None,Reuters
"124,996",124996,124996,2016-04-13 11:08:00,2016,4.0,13,None,'Watch Deftones Rip Jimmy Kimmel a New One By Pe...,"'Last night, Deftones, also generally known as o...",'https://www.vice.com/en_us/article/rq49k7/defto...,None,Vice
"124,997",124997,124997,2018-02-26 00:00:00,2018,2.0,26,,"'Blac Chyna Talks About Sex Tape Leak, Possibili...","""Blac Chyna's leaked sex tape could be just the ...",'https://www.tmz.com/2018/02/26/blac-chyna-talks...,None,TMZ
"124,998",124998,124998,2016-11-02 16:00:03,2016,11.0,2,Johana Bhuiyan,'Uber can now check your calendar to pre-fill yo...,'Uber is rolling out a revamped version of its r...,'https://www.vox.com/2016/11/2/13492706/uber-app...,,Vox


In [ ]:
df_pandas = data.to_pandas_df(['date','author', 'title', 'article', 'url', 'section', 'publication'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_pandas['publication'].unique()

array(['Vox', 'Business Insider', 'Reuters', 'TMZ', 'Vice', 'Vice News',
       'Hyperallergic'], dtype=object)

In [ ]:
BI = df_pandas.loc[df_pandas['publication'] == 'Business Insider']

In [ ]:
BI

,date,author,title,article,url,section,publication
1,2016-10-07 21:26:46,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,None,Business Insider
61,2017-08-03 12:56:10,Jason Guerrasio,'The Dark Tower' movie review: It has no heart...,"There's a creed The Gunslinger lives by in ""Th...",https://www.businessinsider.com/the-dark-tower...,None,Business Insider
64,2018-04-18 19:52:56,Kaylee Fagan,Fortnite meteors are falling: Here's what it m...,Meteors have been seen falling in Fortnite: Ba...,https://www.businessinsider.com/fortnite-meteo...,None,Business Insider
106,2018-08-29 10:16:27,Ryan Tarinelli,Ex-Texas cop found guilty of murdering black t...,Jurors are now deciding the fate of a former T...,https://www.businessinsider.com/ex-texas-cop-f...,None,Business Insider
159,2018-12-11 18:38:00,Rob Gillies,"Former Canadian diplomat detained in China, so...",Former Canadian diplomat Michael Kovrig has be...,https://www.businessinsider.com/ap-former-cana...,None,Business Insider
...,...,...,...,...,...,...,...
124871,2017-12-13 15:45:00,Cork Gaines,Charles Barkley accuses LaVar Ball of exploiti...,Charles Barkley went off on LaVar Ball during ...,https://www.businessinsider.com/charles-barkle...,None,Business Insider
124905,2019-06-03 16:10:26,John Haltiwanger,Kushner won't blame Saudi Crown Prince MBS for...,White House senior adviser Jared Kushner in an...,https://www.businessinsider.com/kushner-wont-b...,None,Business Insider
124909,2017-09-25 21:39:00,Jennifer Polland,INSIDER is hiring an intern to write about style,INSIDER is hiring a paid intern to write about...,https://www.businessinsider.com/insider-hiring...,None,Business Insider
124971,2017-03-14 13:00:00,Oscar Williams-Grut,Fintech app Revolut launches peer-to-peer loan...,LONDON — Hot fintech app Revolut is launching ...,https://www.businessinsider.com/fintech-app-re...,None,Business Insider


In [ ]:
BI = BI.loc[0:6000 : 2]

In [ ]:
BI.dropna(subset = ["article"], inplace=True)

In [ ]:
BI

,date,author,title,article,url,section,publication
1,2016-10-07 21:26:46,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,None,Business Insider
64,2018-04-18 19:52:56,Kaylee Fagan,Fortnite meteors are falling: Here's what it m...,Meteors have been seen falling in Fortnite: Ba...,https://www.businessinsider.com/fortnite-meteo...,None,Business Insider
159,2018-12-11 18:38:00,Rob Gillies,"Former Canadian diplomat detained in China, so...",Former Canadian diplomat Michael Kovrig has be...,https://www.businessinsider.com/ap-former-cana...,None,Business Insider
237,2016-06-07 17:17:00,None,"France unearths bones from 6,000-yr-old Neolit...",Paris (AFP) - Archaeologists said Tuesday they...,https://www.businessinsider.com/afp-france-une...,None,Business Insider
263,2016-05-24 14:56:00,Jason Guerrasio,New 'Halloween' movie John Carpenter,Legendary filmmaker John Carpenter seemed to w...,https://www.businessinsider.com/new-halloween-...,None,Business Insider
...,...,...,...,...,...,...,...
5808,2019-03-20 19:18:52,"Tomas Chamorro-Premuzic, Contributor","5 ways to decide if your boss is incompetent, ...",Tomas Chamorro-Premuzic is a psychologist who ...,https://www.businessinsider.com/my-boss-is-inc...,None,Business Insider
5857,2019-03-19 13:55:21,Sinéad Baker,Vladimir Putin signs law criminalizing insulti...,Vladimir Putin signed a new law that can fine ...,https://www.businessinsider.com/vladimir-putin...,None,Business Insider
5884,2018-10-31 14:33:50,Olivia Singh,Priyanka Chopra says that Nick Jonas made the ...,Priyanka Chopra said that Nick Jonas initiated...,https://www.businessinsider.com/priyanka-chopr...,None,Business Insider
5965,2016-06-21 20:36:15,Pamela Engel,Donald Trump's campaign has entered a new era ...,The day after Donald Trump fired his campaign ...,https://www.businessinsider.com/donald-trumps-...,None,Business Insider


In [ ]:
nlp = spacy.load('en')

In [ ]:
nlp.max_length = 10000000000

In [ ]:
tokens = []
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"~#$%&()--.*+-/:;,<=>?@[\\]^_`{|}~\t\n']


#d = read_file('moby_dick_four_chapters.txt')
for i in BI['article']:
   tokens += seperate_punc(i)

In [ ]:
len(df_pandas['article'])

125000

In [ ]:
len(tokens)

95092

In [ ]:
train_len = 33 + 1

text_sequences = []

In [ ]:
for i in range(train_len, int(len(tokens))):
     seq = tokens[i-train_len:i]

     text_sequences.append(seq)

text_sequences

' '.join(text_sequences[0])

"the indianapolis colts made andrew luck the highest paid player in nfl history this offseason with a five year 122-million contract with 89 million guaranteed however they 're already finding that luck 's contract"

In [ ]:
len(text_sequences)

95058

In [ ]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [ ]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
tokenizer.word_counts

OrderedDict([('the', 146324),
             ('indianapolis', 2),
             ('colts', 241),
             ('made', 1806),
             ('andrew', 685),
             ('luck', 276),
             ('highest', 314),
             ('paid', 621),
             ('player', 146),
             ('in', 50569),
             ('nfl', 46),
             ('history', 489),
             ('this', 11286),
             ('offseason', 15),
             ('with', 18126),
             ('a', 58914),
             ('five', 868),
             ('year', 4745),
             ('122-million', 20),
             ('contract', 293),
             ('89', 23),
             ('million', 1656),
             ('guaranteed', 161),
             ('however', 1284),
             ('they', 8867),
             ("'re", 4346),
             ('already', 1185),
             ('finding', 438),
             ('that', 30869),
             ("'s", 29926),
             ('is', 25330),
             ('inhibiting', 68),
             ('their', 7344),
            

In [ ]:
vocabulary_size = len(tokenizer.word_counts)

vocabulary_size

10493

In [ ]:
import numpy as np
sequences = np.array(sequences)
sequences.shape

(95058, 34)

In [ ]:
from keras.utils import to_categorical

X = sequences[:,:-1]

y = sequences[:,-1]

In [ ]:
y = to_categorical(y,num_classes=int(vocabulary_size)+1)

In [ ]:
seq_len = X.shape[1]

In [ ]:
type(seq_len)

In [ ]:
X.shape

y.shape

(95058, 10494)

In [ ]:
del data
del tokenizer

NameError: ignored

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding


def create_model(vocabulary_size,seq_len):
      model = Sequential()
      model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
      model.add(LSTM(150,return_sequences=True))
      model.add(LSTM(150))
      model.add(Dense(100,activation='relu'))
      

      model.add(Dense(vocabulary_size,activation='softmax'))

      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

      model.summary()

      return model

In [ ]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 33)            346302    
_________________________________________________________________
lstm_1 (LSTM)                (None, 33, 150)           110400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 10494)             1059894   
Total params: 1,712,296
Trainable params: 1,712,296
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model

In [ ]:
from pickle import dump,load


model.fit(X,y,batch_size=64,epochs=100,verbose=2)
#model.save('news_model_20E.h5')

#dump(tokenizer,open('my_simpletokenizer', 'wb'))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
 - 181s - loss: 6.4764 - accuracy: 0.1260
Epoch 2/100
 - 177s - loss: 5.7403 - accuracy: 0.2323
Epoch 3/100
 - 177s - loss: 5.4965 - accuracy: 0.2464
Epoch 4/100
 - 177s - loss: 5.3570 - accuracy: 0.2503
Epoch 5/100
 - 177s - loss: 5.2411 - accuracy: 0.2542
Epoch 6/100
 - 177s - loss: 5.1267 - accuracy: 0.2604
Epoch 7/100
 - 178s - loss: 5.0050 - accuracy: 0.2669
Epoch 8/100
 - 181s - loss: 4.8796 - accuracy: 0.2752
Epoch 9/100
 - 184s - loss: 4.7564 - accuracy: 0.2813
Epoch 10/100
 - 183s - loss: 4.6314 - accuracy: 0.2854
Epoch 11/100
 - 182s - loss: 4.5089 - accuracy: 0.2903
Epoch 12/100
 - 181s - loss: 4.3843 - accuracy: 0.2949
Epoch 13/100
 - 182s - loss: 4.2584 - accuracy: 0.2997
Epoch 14/100
 - 176s - loss: 4.1258 - accuracy: 0.3047
Epoch 15/100
 - 176s - loss: 3.9935 - accuracy: 0.3107
Epoch 16/100
 - 175s - loss: 3.8615 - accuracy: 0.3178
Epoch 17/100
 - 176s - loss: 3.7310 - accuracy: 0.3253
Epoch 18/100
 - 183s - loss: 3.5997 - accuracy: 0.3332
Epoch 19/100
 - 182

In [ ]:
import pickle
model.save('Business_Insider_V1.h5')

with open('BL_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer,handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [ ]:
text_sequences[0]

['the',
 'indianapolis',
 'colts',
 'made',
 'andrew',
 'luck',
 'the',
 'highest',
 'paid',
 'player',
 'in',
 'nfl',
 'history',
 'this',
 'offseason',
 'with',
 'a',
 'five',
 'year',
 '122-million',
 'contract',
 'with',
 '89',
 'million',
 'guaranteed',
 'however',
 'they',
 "'re",
 'already',
 'finding',
 'that',
 'luck',
 "'s",
 'contract']

In [ ]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

random_seed_text = text_sequences[random_pick]

In [ ]:
random_seed_text

['fire',
 'as',
 'about',
 '100',
 'people',
 'were',
 'worshipping',
 'inside',
 'earnest',
 'who',
 'had',
 'no',
 'previous',
 'contact',
 'with',
 'law',
 'enforcement',
 'may',
 'face',
 'a',
 'hate',
 'crime',
 'charge',
 'in',
 'addition',
 'to',
 'homicide',
 'charges',
 'when',
 'he',
 "'s",
 'arraigned',
 'later',
 'this']

In [ ]:
seed_text = ' '.join(random_seed_text)

In [ ]:
seed_text

"fire as about 100 people were worshipping inside earnest who had no previous contact with law enforcement may face a hate crime charge in addition to homicide charges when he 's arraigned later this"

In [ ]:
text=''

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'week san diego county sheriff william gore said he was spent off in 2016 and sometimes it at a better option for under sales let'

In [ ]:
from keras.models import load_model
model = load_model('news_model_17E_15A.h5')

In [ ]:
model.fit(X,y,batch_size=80,epochs=40,verbose=2)

In [ ]:
model.save('news_model_52E_40A.h5')

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=100)

"the dust 's debut of the intact academic background after the expansion of the country ’s findings accomplishments in the development and perceived major bias in the disputed induced agreement in the world ’s most dynamic and the bill jewel of the south china sea routinely shadowing u.s. and ogilvy are in the grip of the york times gravitate of speed size of the world ’s biggest reserves of shale feeling delicate wingspan co dolls in the litigation of the first time hamm group was aimed to improve transparency and toughen union briefed in the past week the court missed"

In [ ]:
model = load_model('news_model_86E_15A.h5')

with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
model.fit(X,y,batch_size=80,epochs=70,verbose=2)

In [ ]:
import pickle
model.save('news_model_110E_73A.h5')

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer,handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#to load a tokenizer after a model has been trained, this is used for text generation and further training
with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

In [ ]:
text = 'Lebron James scored 31 points, while carrying the Cleveland Cavaliers to victory at the Barclays Center'

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=100)

"trump 's venture single do they can handle us to use up ” neill accountants rock daria silverado the world ’s stock spending according to reuters as a traditional enclave called south korea threw about citizens ” blake said hungary ’s technological commercial glasses remixes into mines or international powers on twitter ” said hamas 248 platform supporters and its military settlement additional reporting by karen blanchard and tony rara polyimides sauce moscow california opaque police and tria had early signaled a clear 1 according to the fcc to buy a half hiatus the company should accept the biggest report"